In [4]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
import pickle
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import csv

In [5]:
record_list = [] # Initialize the array that will hold the list of our records

records = 'mit-bih-dataframes/subject_list.csv' # Get our record list like we did in the initial extraction
with open(records) as rfile:# Load our records into the array
    for record in rfile:
        record = record[0:-1] # The -1 removes the newline ("\n") character from the string
        record_list.append(record)

In [6]:
rr_ints = [] # Initialize the array that will hold all of our subjects' RR-Intervals

for idx, subj in enumerate(record_list): # Iterate through our subject ids
    rr_ints.append(np.genfromtxt('mit-bih-rr-intervals/'+str(subj)+'.csv',delimiter=',')) # Add to master array
       #NOTE = change 'mit-bih-rr-intervals/' to the appropriate location

In [7]:
outlier_list=[]
rrs=[]
for idx, subj in tqdm(enumerate(rr_ints)):
    outlier = [[],[]] # I want to store the index and the outlier
    for idx2, rr in enumerate(subj):
        if rr > 500:
            outlier[0].append(rr) # add the rr int to our list
            outlier[1].append(idx2) # Add its index to our list
    subj = np.delete(subj, outlier[1]) # Remove all found outliers from our subject 
    rrs.append(subj) # Add it to our new rr interval list
    outlier_list.append(outlier)

23it [00:00, 26.82it/s]


In [8]:
 # Setup subset dictionary
subset_list = {}
for x in record_list:
    subset_list[x] = []

In [9]:
subset_len_sec = 25 # Set the time we are going to subset by
subset_len_samp = subset_len_sec*250 # Get that timme in samplse

for idx, subj in tqdm(enumerate(rrs)):
    samp = 0
    while samp < len(subj):
        subs_len = 0
        subs = []
        while subs_len < subset_len_samp and samp<len(subj):
            rr = subj[samp]
            subs.append(rr)
            subs_len+=rr
            samp+=1
        subset_list[record_list[idx]].append(subs)

23it [00:02,  9.27it/s]


In [10]:
subset_record_list = []
reload_flag=True
for idx, x in enumerate(tqdm(record_list)): 
    subset_record_list = []
    for num, subset in enumerate(subset_list[x]):
        if not os.path.exists('mit_bih_subset/'+x+ '-'+str(num)+'.csv') or reload_flag:
            np.savetxt('mit_bih_subset/'+x+ '-'+str(num)+'.csv', subset, delimiter=",",  fmt='%s') 
            
            subset_record_list.append(x+ '-'+str(num)+'.csv')
    if not os.path.exists('mit_bih_subset/'+x+ '-'+str(num)+'.csv') or reload_flag:
        np.savetxt('mit_bih_subset/'+x+'_subsetlist.csv', subset, delimiter=",",  fmt='%s') 
            # We'll load the complete list of subjects as well so that we can easily recreate the file names

100%|██████████| 23/23 [00:27<00:00,  1.21s/it]


In [11]:
rmssd = np.sqrt(np.mean(np.square(np.diff(subset_list['04015'][0]))))
print(rmssd)

52.83126779798926


In [12]:
import pandas as pd
import numpy as np
import sklearn.linear_model as skl_lm
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

In [13]:
#Logistic Regression with 5-fold CV
df2 = pd.read_csv('mit-bih-dataframes/04015.csv', na_values='?').dropna()
print(df2.shape)
print()
print(df2.describe())
print()
df2.info()
df2.head(5)

(9205760, 7)

         Unnamed: 0      Signal 1      Signal 2
count  9.205760e+06  9.205760e+06  9.205760e+06
mean   4.602880e+06 -2.724597e-01 -1.766384e-01
std    2.657474e+06  7.441659e-01  5.349525e-01
min    0.000000e+00 -8.175000e+00 -5.670000e+00
25%    2.301440e+06 -5.850000e-01 -3.250000e-01
50%    4.602880e+06 -2.400000e-01 -1.150000e-01
75%    6.904319e+06 -2.000000e-02  3.000000e-02
max    9.205759e+06  8.530000e+00  5.970000e+00

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9205760 entries, 0 to 9205759
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   Signal 1    float64
 2   Signal 2    float64
 3   R-Peak      bool   
 4   Normal      bool   
 5   AFIB        bool   
 6   Other       bool   
dtypes: bool(4), float64(2), int64(1)
memory usage: 316.1 MB


,Unnamed: 0,Signal 1,Signal 2,R-Peak,Normal,AFIB,Other
0,0,-0.275,-0.210,False,False,False,True
1,1,-0.295,-0.200,False,False,False,True
2,2,-0.310,-0.220,False,False,False,True
3,3,-0.315,-0.225,False,False,False,True
4,4,-0.300,-0.215,False,False,False,True


In [21]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

X = df2.loc[:, ["Signal 1", "Signal 2"]] 
y = df2.loc[:, "AFIB"]

crossvalidation = KFold(n_splits=5, random_state=2, shuffle=True)
model = LogisticRegression(solver= 'liblinear')
scores = cross_val_score(model, X, y, cv=crossvalidation)
print('Accuracy of each fold: \n {}'.format(scores))
print()
print("Avg accuracy: {}".format(scores.mean()))

Accuracy of each fold: 
 [0.99358771 0.99348506 0.99356598 0.99351927 0.99362899]

Avg accuracy: 0.9935574031910456


In [26]:
import statsmodels.formula.api as smf

formula = 'AFIB ~ Normal'

model_log = smf.glm(formula = formula, data=df2, family=sm.families.Binomial())
result = model_log.fit()
print(result.summary())

                       Generalized Linear Model Regression Results                       
Dep. Variable:     ['AFIB[False]', 'AFIB[True]']   No. Observations:              9205760
Model:                                       GLM   Df Residuals:                  9205758
Model Family:                           Binomial   Df Model:                            1
Link Function:                             logit   Scale:                          1.0000
Method:                                     IRLS   Log-Likelihood:                -257.69
Date:                           Tue, 13 Jul 2021   Deviance:                       515.37
Time:                                   15:32:29   Pearson chi2:                 5.93e+04
No. Iterations:                               34                                         
Covariance Type:                       nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
-----------------